In [1]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
from dataclasses import dataclass, field, fields
import pack_cuda
import pack_vis
import pack_cost
import copy
import time
import pack_ga
from IPython.display import HTML, display, clear_output
kgs.set_float32(True)


vast
vast
stop final relax at some point


In [2]:
%load_ext pyinstrument

In [3]:
importlib.reload(pack_ga)
fastMode = False

# fast for now
#runner.population_size = 1
#runner.selection_size = [0]
#runner.n_generations = 500

# structural
ratios = np.arange(0.3,1.01,0.01)
res = []
for r in ratios:
    print(r)
    runner = pack_ga.GA()
    runner.N_trees_to_do = np.array([2])
    runner.population_size = 1000
    runner.plot_fitness_predictors = False
    runner.plot_diversity_matrix = False
    runner.plot_champions = False
    runner.initializer.base_solution = kgs.SolutionCollectionLatticeFixed()
    runner.initializer.base_solution.aspect_ratios = cp.array([1/r], dtype=kgs.dtype_cp)
    runner.initializer.jiggler.duration_init = 0.
    runner.initializer.jiggler.duration_compact /= 100000
    runner.initializer.jiggler.duration_final = 0.
    runner.initializer.jiggler.plot_interval = None
    runner.initializer.jiggler.n_rounds = 1
    runner.rough_relaxers[0].plot_interval =None
    runner.rough_relaxers[0].n_iterations//=5
    runner.fine_relaxers[0].plot_interval=None
    runner.rough_relaxers[0].cost.costs.pop(1)
    runner.fine_relaxers[0].cost.costs.pop(1)
    runner.fine_relaxers[0].n_iterations//=5
    runner.n_generations = 5
    runner.fitness_cost.costs.pop(1)
    kgs.debugging_mode = 1
    kgs.profiling=False
    new_moves = []
    for i in [1,2]:
        new_moves.append(runner.move.moves[i])
    runner.move.moves = new_moves
    #print(runner.move.moves)
    #runner.rough_relaxers = []
    #runner.fine_relaxers = []
    runner.run()
    res.append(runner)

stop final relax at some point
0.3
Generation 0, Trees 2, Best cost: 0.00950821, Est: 0.47541064, h: 0.529780
[[0.52978045]]
Generation 1, Trees 2, Best cost: 0.00950820, Est: 0.47541019, h: 0.529752
[[0.5297516]]
Generation 2, Trees 2, Best cost: 0.00950820, Est: 0.47540990, h: 0.529712
[[0.52971154]]
Generation 3, Trees 2, Best cost: 0.00950820, Est: 0.47540990, h: 0.529731
[[0.5297314]]
Generation 4, Trees 2, Best cost: 0.00950820, Est: 0.47540987, h: 0.529726
[[0.52972573]]
0.31
Generation 0, Trees 2, Best cost: 0.00920636, Est: 0.46031806, h: 0.530049
[[0.53004885]]
Generation 1, Trees 2, Best cost: 0.00920635, Est: 0.46031758, h: 0.530005
[[0.53000474]]
Generation 2, Trees 2, Best cost: 0.00920635, Est: 0.46031755, h: 0.529998
[[0.529998]]
Generation 3, Trees 2, Best cost: 0.00920635, Est: 0.46031737, h: 0.529992
[[0.5299917]]
Generation 4, Trees 2, Best cost: 0.00920635, Est: 0.46031737, h: 0.529992
[[0.5299917]]
0.32
Generation 0, Trees 2, Best cost: 0.00892310, Est: 0.44615483

In [ ]:
res_pop = []
for r in res:
    pop = copy.deepcopy(r.populations[0])
    pop.select_ids([np.argmin(pop.fitness)])
    res_pop.append(pop.configuration)
pop = res_pop[0]
for r in res_pop[1:]:
    pop.merge(r)
kgs.dill_save(kgs.code_dir + '../res/packings.pickle', pop)